In [14]:
import cubewalkers as cw
import cupy as cp
from copy import deepcopy

In [15]:
IMPORT_RULES_FROM_FILES = True
cc_models_dir = './models/cell_collective/'
if IMPORT_RULES_FROM_FILES:
    from os import listdir
    
    sync_models = {}
    for fname in listdir(cc_models_dir):
        with open(cc_models_dir+fname) as rulefile:
            name = fname.strip('.txt')
            rules = rulefile.read()
            sync_models[name]=cw.Model(rules)
else:
    from cana.datasets.bio import load_all_cell_collective_models
    def cell_collective_models():
        return {BN.name:cw.Model(cw.conversions.network_rules_from_cana(BN)) 
                for BN in load_all_cell_collective_models()}
    sync_models = cell_collective_models()
    for name,model in sync_models.items():
        with open(cc_models_dir+name+'.txt','w') as rulefile:
            rulefile.write(model.rules)

total_models = len(sync_models)
async_models = deepcopy(sync_models)

In [16]:
def simulate_models(models, sync=True ,W=2500):
    convergence_measures = {}
    for model_idx, (model_name, model) in enumerate(models.items()):
        model.n_walkers = W
        N = model.n_variables
        T = N**2 + 5 * N
        T_window = N * 5
        model.n_time_steps = T

        print(f"Simulating Model {model_name} ({W=},{T=},{N=}). . .")
        if sync:
            model.simulate_ensemble(T_window=T_window,
                                    averages_only=True,
                                    maskfunction=cw.update_schemes.synchronous,
                                    threads_per_block=(16, 16))
        else:
            model.simulate_ensemble(T_window=T_window,
                                    averages_only=True,
                                    maskfunction=cw.update_schemes.asynchronous,
                                    threads_per_block=(16, 16))
            
        tw1 = model.trajectories[0:2*N]
        tw2 = model.trajectories[N:3*N]
        tw3 = model.trajectories[2*N:4*N]
        tw4 = model.trajectories[3*N:]
        convergence_measures[model_name] = max([
            cp.max(cp.abs(tw1.mean(axis=0) - tw2.mean(axis=0))),    
            cp.max(cp.abs(tw1.mean(axis=0) - tw3.mean(axis=0))),
            cp.max(cp.abs(tw1.mean(axis=0) - tw4.mean(axis=0))),
            cp.max(cp.abs(tw2.mean(axis=0) - tw3.mean(axis=0))),
            cp.max(cp.abs(tw2.mean(axis=0) - tw4.mean(axis=0))),
            cp.max(cp.abs(tw3.mean(axis=0) - tw4.mean(axis=0))),
            ])
        print(f"Progress: {(model_idx+1)}/{total_models},\t maximum difference: {convergence_measures[model_name]}")
    return convergence_measures

In [17]:
sync_convergence_measures=simulate_models(sync_models,sync=True)

Simulating Model T-LGL Survival Network 2011 Reduced Network (W=2500,T=414,N=18). . .
Progress: 1/74,	 maximum difference: 0.0
Simulating Model TOL Regulatory Network (W=2500,T=696,N=24). . .
Progress: 2/74,	 maximum difference: 0.0
Simulating Model B cell differentiation (W=2500,T=594,N=22). . .
Progress: 3/74,	 maximum difference: 0.0
Simulating Model Death Receptor Signaling (W=2500,T=924,N=28). . .
Progress: 4/74,	 maximum difference: 0.0012785714285714622
Simulating Model HCC1954 Breast Cell Line Short-term ErbB Network (W=2500,T=336,N=16). . .
Progress: 5/74,	 maximum difference: 0.0
Simulating Model IL-1 Signaling (W=2500,T=14514,N=118). . .
Progress: 6/74,	 maximum difference: 0.0
Simulating Model T-LGL Survival Network 2011 (W=2500,T=3900,N=60). . .
Progress: 7/74,	 maximum difference: 0.0
Simulating Model Mammalian Cell Cycle (W=2500,T=500,N=20). . .
Progress: 8/74,	 maximum difference: 0.0
Simulating Model Signaling in Macrophage Activation (W=2500,T=104646,N=321). . .
Progr

In [18]:
async_convergence_measures=simulate_models(async_models,sync=False)

Simulating Model T-LGL Survival Network 2011 Reduced Network (W=2500,T=414,N=18). . .
Progress: 1/74,	 maximum difference: 0.0035888888888888915
Simulating Model TOL Regulatory Network (W=2500,T=696,N=24). . .
Progress: 2/74,	 maximum difference: 0.0
Simulating Model B cell differentiation (W=2500,T=594,N=22). . .
Progress: 3/74,	 maximum difference: 0.0
Simulating Model Death Receptor Signaling (W=2500,T=924,N=28). . .
Progress: 4/74,	 maximum difference: 9.285714285711677e-05
Simulating Model HCC1954 Breast Cell Line Short-term ErbB Network (W=2500,T=336,N=16). . .
Progress: 5/74,	 maximum difference: 0.0
Simulating Model IL-1 Signaling (W=2500,T=14514,N=118). . .
Progress: 6/74,	 maximum difference: 0.0
Simulating Model T-LGL Survival Network 2011 (W=2500,T=3900,N=60). . .
Progress: 7/74,	 maximum difference: 0.0026233333333333326
Simulating Model Mammalian Cell Cycle (W=2500,T=500,N=20). . .
Progress: 8/74,	 maximum difference: 0.0
Simulating Model Signaling in Macrophage Activatio

In [20]:
print(f'{max(sync_convergence_measures.values())}')
print(f'{max(async_convergence_measures.values())}')

0.08710000000000007
0.03733333333333344


In [21]:
for model_name, model in [(k,async_models[k]) for k in sorted(async_models)]:
    print(model_name+',asynchronous,'+','.join(map(lambda x: str(cp.round(x,3)),cp.mean(model.trajectories,axis=0))))

Apoptosis Network,asynchronous,0.528,0.298,0.241,0.498,0.498,0.498,0.191,0.077,0.498,0.187,0.162,0.242,0.704,0.257,0.19,0.528,0.585,0.518,0.299,0.519,0.242,0.528,0.498,0.498,0.498,0.291,0.41,0.498,0.498,0.518,0.519,0.519,0.583,0.498,0.167,0.259,0.443,0.528,0.19,0.498,0.519
Arabidopsis thaliana Cell Cycle,asynchronous,0.279,0.253,0.42,0.602,0.467,0.545,0.541,0.454,0.674,0.269,0.439,0.389,0.358,0.577
Aurora Kinase A in Neuroblastoma,asynchronous,0.449,0.449,0.437,0.236,0.551,0.438,0.551,0.551,0.551,0.551,0.44,0.113,0.449,0.551,0.228,0.449,0.439,0.551,0.441,0.489,0.504,0.491,0.501
B bronchiseptica and T retortaeformis coinfection,asynchronous,0.0,0.0,0.122,0.122,0.0,0.0,0.141,0.122,0.142,0.122,0.0,0.0,0.0,0.0,0.122,0.0,0.493,0.122,0.493,0.142,0.122,0.961,0.622,0.622,0.142,0.0,0.0,0.182,0.788,0.622,0.0,0.966,0.0,0.622,0.0,0.0,0.122,0.122,0.0,0.0,0.961,0.142,0.0,0.804,0.122,0.0,0.142,0.928,0.441,0.122,0.804,0.0,0.493
B cell differentiation,asynchronous,0.483,0.504,0.507,0.481,0.505,0.201,0.

In [22]:
for model_name, model in [(k,sync_models[k]) for k in sorted(sync_models)]:
    print(model_name+',synchronous,'+','.join(map(lambda x: str(cp.round(x,3)),cp.mean(model.trajectories,axis=0))))

Apoptosis Network,synchronous,0.266,0.139,0.418,0.491,0.491,0.491,0.209,0.013,0.491,0.209,0.135,0.28,0.611,0.133,0.208,0.266,0.621,0.449,0.139,0.495,0.28,0.266,0.491,0.491,0.491,0.242,0.379,0.491,0.491,0.449,0.495,0.495,0.621,0.491,0.135,0.181,0.439,0.266,0.162,0.491,0.495
Arabidopsis thaliana Cell Cycle,synchronous,0.263,0.181,0.457,0.53,0.559,0.447,0.351,0.544,0.734,0.286,0.553,0.457,0.35,0.647
Aurora Kinase A in Neuroblastoma,synchronous,0.18,0.214,0.542,0.28,0.797,0.555,0.786,0.797,0.786,0.786,0.54,0.279,0.185,0.786,0.28,0.203,0.356,0.82,0.539,0.509,0.496,0.499,0.493
B bronchiseptica and T retortaeformis coinfection,synchronous,0.0,0.0,0.015,0.021,0.0,0.0,0.028,0.015,0.028,0.021,0.0,0.0,0.0,0.0,0.021,0.0,0.505,0.021,0.505,0.028,0.021,0.999,0.693,0.692,0.028,0.0,0.0,0.15,0.738,0.692,0.0,0.999,0.0,0.693,0.0,0.0,0.021,0.021,0.0,0.0,0.999,0.028,0.0,0.738,0.021,0.0,0.028,0.998,0.495,0.027,0.738,0.0,0.505
B cell differentiation,synchronous,0.51,0.484,0.488,0.507,0.514,0.095,0.484,0.488,0